In [4]:
pip install pandas

In [1]:
import pandas as pd
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# Carrega o arquivo CSV no Pandas
caminho_arquivo = 'preços.csv'
dados = pd.read_csv(caminho_arquivo, sep=';')

# Converte a coluna 'Data da Coleta' para o formato de data
dados['Data da Coleta'] = pd.to_datetime(dados['Data da Coleta'])

# Inicializa o aplicativo Dash
app = dash.Dash(__name__)

# Layout do aplicativo
app.layout = html.Div([
    # Filtros
    html.Label('Ano:'),
    dcc.Dropdown(
        id='filtro-ano',
        options=[{'label': str(ano), 'value': ano} for ano in dados['Data da Coleta'].dt.year.unique()],
        value='Todos',
        multi=False
    ),

    html.Label('Mês:'),
    dcc.Dropdown(
        id='filtro-mes',
        options=[{'label': str(mes), 'value': mes} for mes in dados['Data da Coleta'].dt.month.unique()],
        value='Todos',
        multi=False
    ),

    html.Label('Sigla Estado:'),
    dcc.Dropdown(
        id='filtro-estado',
        options=[{'label': estado, 'value': estado} for estado in dados['Estado - Sigla'].unique()],
        value='Todos',
        multi=False
    ),

    html.Label('Região:'),
    dcc.Dropdown(
        id='filtro-regiao',
        options=[{'label': regiao, 'value': regiao} for regiao in dados['Regiao - Sigla'].unique()],
        value='Todos',
        multi=False
    ),

    html.Label('Produtos:'),
    dcc.Dropdown(
        id='filtro-produto',
        options=[{'label': produto, 'value': produto} for produto in dados['Produto'].unique()],
        value='Todos',
        multi=False
    ),

    # Gráficos
    dcc.Graph(id='grafico-media-estado'),
    dcc.Graph(id='grafico-media-municipio')
])

# Callbacks para atualizar os gráficos com base nos filtros selecionados
@app.callback(
    [Output('grafico-media-estado', 'figure'),
     Output('grafico-media-municipio', 'figure')],
    [Input('filtro-ano', 'value'),
     Input('filtro-mes', 'value'),
     Input('filtro-estado', 'value'),
     Input('filtro-regiao', 'value'),
     Input('filtro-produto', 'value')]
)
def update_graficos(ano, mes, estado, regiao, produto):
    # Aplica os filtros aos dados
    dados_filtrados = dados.copy()
    if ano != 'Todos':
        dados_filtrados = dados_filtrados[dados_filtrados['Data da Coleta'].dt.year == int(ano)]
    if mes != 'Todos':
        dados_filtrados = dados_filtrados[dados_filtrados['Data da Coleta'].dt.month == int(mes)]
    if estado != 'Todos':
        dados_filtrados = dados_filtrados[dados_filtrados['Sigla Estado'] == estado]
    if regiao != 'Todos':
        dados_filtrados = dados_filtrados[dados_filtrados['Região'] == regiao]
    if produto != 'Todos':
        dados_filtrados = dados_filtrados[dados_filtrados['Produtos'] == produto]

    # Gráfico de média por Sigla Estado
    grafico_estado = px.bar(
        dados_filtrados.groupby('Sigla Estado')['Valor Venda'].mean().reset_index(),
        x='Sigla Estado', y='Valor Venda', title='Média do Valor de Venda por Sigla Estado'
    )

    # Gráfico de média por Município
    grafico_municipio = px.scatter(
        dados_filtrados.groupby('Municipio')['Valor Venda'].mean().reset_index(),
        x='Municipio', y='Valor Venda', title='Média do Valor de Venda por Município'
    )

    return grafico_estado, grafico_municipio

# Executa o aplicativo
if __name__ == '__main__':
    app.run_server(debug=True)


ModuleNotFoundError: No module named 'pandas'